In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data" , one_hot = True)
## 每个批次的大小
batch_size = 100

# 计算一共有多少批次
n_batch = mnist.train.num_examples  // batch_size

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def residual_block(x, output_channel):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2, 2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,
                             output_channel,
                             (3,3),
                             strides = strides,
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv1')
    conv2 = tf.layers.conv2d(conv1,
                             output_channel,
                             (3, 3),
                             strides = (1, 1),
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv2')
    if increase_dim:
        # [None, image_width, image_height, channel] -> [,,,channel*2]
        pooled_x = tf.layers.average_pooling2d(x,
                                               (2, 2),
                                               (2, 2),
                                               padding = 'valid')
        padded_x = tf.pad(pooled_x,
                          [[0,0],
                           [0,0],
                           [0,0],
                           [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    output_x = conv2 + padded_x
    return output_x

In [4]:
def res_net(x, 
            num_residual_blocks, 
            num_filter_base,
            class_num):
    
    num_subsampling = len(num_residual_blocks)
    layers = []
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x, 
                                 num_filter_base,
                                 (3, 3),
                                 strides = (1, 1),
                                 padding = 'same',
                                 activation = tf.nn.relu,
                                 name = 'conv0')
        layers.append(conv0)
    # eg:num_subsampling = 4, sample_id = [0,1,2,3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],
                    num_filter_base * (2 ** sample_id))
                layers.append(conv)
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]
    with tf.variable_scope('fc'):
        # layer[-1].shape : [None, width, height, channel]
        # kernal_size: image_width, image_height
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]

In [5]:
# 定义placeholder
x = tf.placeholder(tf.float32 , [None , 784])
y = tf.placeholder(tf.float32 , [None , 10])
lr = tf.Variable(0.001,dtype=tf.float32)

# 改变x的格式转为4D的向量 [batch , in_height , in_width , in_channels]
x_image = tf.reshape(x , [-1,28,28,1])

# 得到预测值
prediction = res_net(x_image, [2,3,2], 32, 10)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y , logits=prediction))

# 使用梯度下降
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(prediction,1))

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.


In [6]:
# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):
        sess.run(tf.assign(lr,0.0005*(0.95**epoch)))
        for batch in range(n_batch):
            batch_xs , batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step , feed_dict={
                x:batch_xs,
                y:batch_ys
            })
        acc = sess.run(accuracy , feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels
        })
        print("Iter" + str(epoch) + ",Testing Accuracy : " + str(acc))

'''效果会更好，但是计算时间慢'''

Iter0,Testing Accuracy : 0.9778
Iter1,Testing Accuracy : 0.9855
Iter2,Testing Accuracy : 0.9889
Iter3,Testing Accuracy : 0.9889
Iter4,Testing Accuracy : 0.9936
Iter5,Testing Accuracy : 0.9916
Iter6,Testing Accuracy : 0.9932
Iter7,Testing Accuracy : 0.9918
Iter8,Testing Accuracy : 0.9939
Iter9,Testing Accuracy : 0.993
Iter10,Testing Accuracy : 0.9939
Iter11,Testing Accuracy : 0.9942
Iter12,Testing Accuracy : 0.9947
Iter13,Testing Accuracy : 0.9914
Iter14,Testing Accuracy : 0.9945
Iter15,Testing Accuracy : 0.9938
Iter16,Testing Accuracy : 0.9938
Iter17,Testing Accuracy : 0.9919
Iter18,Testing Accuracy : 0.9936
Iter19,Testing Accuracy : 0.9917


'效果会更好，但是计算时间慢'